In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.utils import to_categorical

2024-08-04 22:36:54.806641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 22:36:54.806791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 22:36:54.963398: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [49]:
import string, random

In [50]:
# Function to convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Function to convert binary to DNA
def binary_to_dna(binary):
    return ''.join(dna_encoding[binary[i:i+2]] for i in range(0, len(binary), 2))

# Function to convert DNA to hexadecimal
def dna_to_hex(dna):
    return ''.join(format(ord(base), '02x') for base in dna)

# Functions for encryption and decryption
def encrypt(plaintext, key):
    binary_text = text_to_binary(plaintext)
    input_data = binary_text + key
    input_data = to_categorical([int(bit) for bit in input_data], num_classes=2)
    input_data = input_data.reshape(1, input_data.shape[0], 2)
    
    encrypted = enc_model.predict(input_data)
    encrypted_hex = ''.join([format(np.argmax(char), '01x') for char in encrypted[0]])
    return encrypted_hex

def decrypt(ciphertext, key):
    cipher_one_hot = to_categorical([int(char, 16) for char in ciphertext], num_classes=16)
    key_one_hot = to_categorical([int(bit) for bit in key], num_classes=2)
    dec_input = np.concatenate([cipher_one_hot, key_one_hot], axis=-1)
    dec_input = dec_input.reshape(1, dec_input.shape[0], dec_input.shape[1])
    
    decrypted_binary = dec_model.predict(dec_input)[0]
    decrypted_binary = ['1' if bit > 0.5 else '0' for bit in decrypted_binary]
    decrypted_text = ''.join([chr(int(''.join(decrypted_binary[i:i+8]), 2)) for i in range(0, len(decrypted_binary), 8)])
    return decrypted_text.rstrip('\x00')  # Remove any padding

def generate_random_string(length=13):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

In [53]:
# DNA encoding
dna_encoding = {'00': 'A', '01': 'C', '10': 'G', '11': 'T'}
dna_decoding = {v: k for k, v in dna_encoding.items()}

# Generate sample data ( all of same length 13)
# plaintexts = ["hello, world!", "cryptography!", "deep learning"]
# Generate 1000 random strings
plaintexts = [generate_random_string(13) for _ in range(1000)]
keys = [np.random.randint(0, 2, 64).astype(str).tolist() for _ in range(len(plaintexts))]
keys = [''.join(key) for key in keys]

# Ensure all sequences are of the same length
max_length = max(len(text_to_binary(text)) + len(key) for text, key in zip(plaintexts, keys))

# Create training data for encryption
X_enc_train = []
y_enc_train = []

for plaintext, key in zip(plaintexts, keys):
    binary_text = text_to_binary(plaintext)
    dna_text = binary_to_dna(binary_text)
#     encrypted_dna = ''.join(chr((ord(a) ^ ord(b)) % 4 + ord('A')) for a, b in zip(dna_text, key))
    encrypted_hex = dna_to_hex(dna_text)
    
#     print(len(binary_text), ' ', len(key), ' ', len(encrypted_hex))
#     print(binary_text, ' ', key, ' ', dna_text, ' ', encrypted_hex)
#     print(len(binary_text), ' ', len(key), ' ', len(dna_text), ' ', len(encrypted_hex))
    
    # Pad sequences to ensure uniform length
    padded_binary_text = binary_text.ljust(max_length - len(key), '0')
    padded_key = key.ljust(max_length - len(binary_text), '0')
    padded_encrypted_hex = encrypted_hex.ljust(max_length, '0')
    
#     print(len(padded_binary_text), ' ', len(padded_key), ' ', len(padded_encrypted_hex))
    X_enc_train.append(list(padded_binary_text + padded_key))
    y_enc_train.append(list(padded_encrypted_hex))

X_enc_train = np.array(X_enc_train)
y_enc_train = np.array(y_enc_train)

# Convert to one-hot encoding
X_enc_train = to_categorical(X_enc_train, num_classes=2)
y_enc_train = to_categorical([int(char, 16) for hex_str in y_enc_train for char in hex_str], num_classes=16)
# Reshape y_enc_train
y_enc_train = y_enc_train.reshape(len(plaintexts), 168, 16)

In [54]:
X_enc_train.shape, y_enc_train.shape

((1000, 168, 2), (1000, 168, 16))

In [58]:
# Create and train the encryption model
enc_model = Sequential([
    LSTM(128, input_shape=(X_enc_train.shape[1], 2), return_sequences=True),
    Dense(16, activation='softmax')
])

enc_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
enc_model.fit(X_enc_train, y_enc_train, epochs=15, batch_size=3)

Epoch 1/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 26s 72ms/step - accuracy: 0.4371 - loss: 1.5907
Epoch 2/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.7481 - loss: 0.6283
Epoch 3/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - accuracy: 0.7641 - loss: 0.5036
Epoch 4/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.7678 - loss: 0.4771
Epoch 5/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.7687 - loss: 0.4680
Epoch 6/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.7714 - loss: 0.4599
Epoch 7/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.7808 - loss: 0.4503
Epoch 8/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.7738 - loss: 0.4658
Epoch 9/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 24s 73ms/step - accuracy: 0.7878 - loss: 0.4370
Epoch 10/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - accuracy: 0.8359 - loss: 0.3692
Epoch 11/15
334/334 ━━━━━━━━━━━━━━━━━━━━ 25s 73ms/step - accuracy: 0.8357 - loss: 0.3624
Epoch 12/15
334/334 ━━━━━━━━━━

In [68]:
# Test the models
test_plaintext = "dewudheuifw"
test_key = ''.join(np.random.randint(0, 2, 64).astype(str))

encrypted = encrypt(test_plaintext, test_key)
print(f"Plaintext: {test_plaintext}")
print(f"Encrypted (hex): {encrypted}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Plaintext: dewudheuifw
Encrypted (hex): 43471574434743434354435443544343435743414357574143574343435443434357574343574357435443544741415443434344000000000000000000000000000000000000000000000000


In [30]:
# Create training data for decryption
X_dec_train = []
y_dec_train = []

for ciphertext, key, plaintext in zip(y_enc_train, keys, plaintexts):
    # Combine ciphertext and key
    key_one_hot = to_categorical([int(bit) for bit in key], num_classes=2)
    X_dec_train.append(np.concatenate([ciphertext, key_one_hot], axis=-1))
    
    # Convert plaintext to binary
    plaintext_binary = text_to_binary(plaintext)
    y_dec_train.append([int(bit) for bit in plaintext_binary])

X_dec_train = np.array(X_dec_train)
y_dec_train = np.array(y_dec_train)

# Create and train the decryption model
max_plaintext_binary_length = max(len(text_to_binary(text)) for text in plaintexts)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [ ]:
dec_model = Sequential([
    LSTM(128, input_shape=(X_dec_train.shape[1], X_dec_train.shape[2]), return_sequences=True),
    Dense(64, activation='relu'),
    Dense(max_plaintext_binary_length, activation='sigmoid')
])

dec_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dec_model.fit(X_dec_train, y_dec_train, epochs=100, batch_size=1)


# Test the models
test_plaintext = "Test"
test_key = ''.join(np.random.randint(0, 2, 64).astype(str))

encrypted = encrypt(test_plaintext, test_key)
print(f"Plaintext: {test_plaintext}")
print(f"Encrypted (hex): {encrypted}")

decrypted = decrypt(encrypted, test_key)
print(f"Decrypted: {decrypted}")